In [345]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk.corpus
from nltk.stem.snowball import SnowballStemmer
from sklearn.cross_validation import train_test_split
from ml_metrics import rmsle
import xgboost as xgb
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import datasets, linear_model
import scipy.sparse as sps
from scipy.sparse import coo_matrix, hstack, vstack, csr_matrix
from scipy import io
from datetime import datetime
import gc
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [346]:
def evalerror(preds, dtrain):

    labels = dtrain.get_label()
    assert len(preds) == len(labels)
    labels = labels.tolist()
    preds = preds.tolist()
    terms_to_sum = [(math.log(labels[i] + 1) - math.log(max(0,preds[i]) + 1))
                    ** 2.0 for i,pred in enumerate(labels)]
    return 'error', (sum(terms_to_sum) * (1.0/len(preds))) ** 0.5

In [347]:
train = pd.read_csv('../input/train.csv', usecols=['Semana', 'Cliente_ID', 'Producto_ID', 'Demanda_uni_equil'])
test = pd.read_csv('../input/test.csv', usecols=['id', 'Semana', 'Cliente_ID', 'Producto_ID'])
train.tail()

,Semana,Cliente_ID,Producto_ID,Demanda_uni_equil
74180459,9,4528866,32873,4
74180460,9,4528866,34226,4
74180461,9,4528866,45112,4
74180462,9,4547943,40217,0
74180463,9,4708097,43159,1


In [348]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74180464 entries, 0 to 74180463
Data columns (total 4 columns):
Semana               int64
Cliente_ID           int64
Producto_ID          int64
Demanda_uni_equil    int64
dtypes: int64(4)
memory usage: 2.2 GB


In [349]:
test.head()

,id,Semana,Cliente_ID,Producto_ID
0,0,11,4639078,35305
1,1,11,4705135,1238
2,2,10,4549769,32940
3,3,11,4717855,43066
4,4,11,966351,1277


In [350]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999251 entries, 0 to 6999250
Data columns (total 4 columns):
id             int64
Semana         int64
Cliente_ID     int64
Producto_ID    int64
dtypes: int64(4)
memory usage: 213.6 MB


In [351]:
grouping = ['Semana', 'Cliente_ID', 'Producto_ID']
train_group = train.groupby(grouping,as_index=False)['Demanda_uni_equil'].sum()

In [352]:
train_group.tail()

,Semana,Cliente_ID,Producto_ID,Demanda_uni_equil
74013018,9,2015152015,1278,20
74013019,9,2015152015,2233,3
74013020,9,2015152015,2665,10
74013021,9,2015152015,4280,8
74013022,9,2015152015,31717,3


In [353]:
train_group.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74013023 entries, 0 to 74013022
Data columns (total 4 columns):
Semana               int64
Cliente_ID           int64
Producto_ID          int64
Demanda_uni_equil    int64
dtypes: int64(4)
memory usage: 2.8 GB


In [354]:
train_group.head()

,Semana,Cliente_ID,Producto_ID,Demanda_uni_equil
0,3,26,1182,39
1,3,26,4767,42
2,3,26,31393,20
3,3,26,31690,42
4,3,26,32962,3


In [355]:
train_cliprod = train_group
train_cliprod["cliprod"] = train_cliprod["Cliente_ID"].map(str) + ' ' + train_cliprod["Producto_ID"].map(str)

In [356]:
test_cliprod = test
test_cliprod["cliprod"] = test_cliprod["Cliente_ID"].map(str) + ' ' + test_cliprod["Producto_ID"].map(str)

In [357]:
train_cliprod.head()

,Semana,Cliente_ID,Producto_ID,Demanda_uni_equil,cliprod
0,3,26,1182,39,26 1182
1,3,26,4767,42,26 4767
2,3,26,31393,20,26 31393
3,3,26,31690,42,26 31690
4,3,26,32962,3,26 32962


In [358]:
test_cliprod.head()

,id,Semana,Cliente_ID,Producto_ID,cliprod
0,0,11,4639078,35305,4639078 35305
1,1,11,4705135,1238,4705135 1238
2,2,10,4549769,32940,4549769 32940
3,3,11,4717855,43066,4717855 43066
4,4,11,966351,1277,966351 1277


In [359]:
grouping = ['Semana', 'cliprod']
train_cliprod = train_cliprod.groupby(grouping,as_index=False)['Demanda_uni_equil'].sum()


In [360]:
train_cliprod.head()

,Semana,cliprod,Demanda_uni_equil
0,3,100000 43274,1
1,3,1000001 1220,2
2,3,1000001 1240,11
3,3,1000001 1242,2
4,3,1000001 1250,5


In [361]:
train_cliprod.tail()

,Semana,cliprod,Demanda_uni_equil
74013018,9,9999999 43066,1
74013019,9,9999999 43069,23
74013020,9,9999999 43084,1
74013021,9,9999999 43274,11
74013022,9,9999999 43285,45


In [362]:
s3 = train_cliprod[train_cliprod.Semana == 3].drop('Semana', axis=1).set_index('cliprod')
s4 = train_cliprod[train_cliprod.Semana == 4].drop('Semana', axis=1).set_index('cliprod')
s5 = train_cliprod[train_cliprod.Semana == 5].drop('Semana', axis=1).set_index('cliprod')
s6 = train_cliprod[train_cliprod.Semana == 6].drop('Semana', axis=1).set_index('cliprod')
s7 = train_cliprod[train_cliprod.Semana == 7].drop('Semana', axis=1).set_index('cliprod')
s8 = train_cliprod[train_cliprod.Semana == 8].drop('Semana', axis=1).set_index('cliprod')
s9 = train_cliprod[train_cliprod.Semana == 9].drop('Semana', axis=1).set_index('cliprod')

t10 = test_cliprod[test_cliprod.Semana == 10].drop('Semana', axis=1).set_index('cliprod')
t11 = test_cliprod[test_cliprod.Semana == 11].drop('Semana', axis=1).set_index('cliprod')

In [363]:
s3.head()

,Demanda_uni_equil
cliprod,
100000 43274,1
1000001 1220,2
1000001 1240,11
1000001 1242,2
1000001 1250,5


In [364]:
t10.head()

,id,Cliente_ID,Producto_ID
cliprod,,,
4549769 32940,2,4549769,32940
4414012 35305,7,4414012,35305
397854 1240,8,397854,1240
4387996 2233,11,4387996,2233
4446449 1240,13,4446449,1240


In [365]:
s6["d2"]=s4['Demanda_uni_equil']
s7["d2"]=s5['Demanda_uni_equil']
s8["d2"]=s6['Demanda_uni_equil']
s9["d2"]=s7['Demanda_uni_equil']

s6["d3"]=s3['Demanda_uni_equil']
s7["d3"]=s4['Demanda_uni_equil']
s8["d3"]=s5['Demanda_uni_equil']
s9["d3"]=s6['Demanda_uni_equil']

t10["d2"]=s8['Demanda_uni_equil']
t11["d2"]=s9['Demanda_uni_equil']

t10["d3"]=s7['Demanda_uni_equil']
t11["d3"]=s8['Demanda_uni_equil']

In [366]:
s6.head()

,Demanda_uni_equil,d2,d3
cliprod,,,
1000001 1309,2,NaN,2.0
1000001 30572,4,NaN,NaN
1000001 43285,8,NaN,NaN
1000004 1238,1,NaN,1.0
1000004 1250,0,2.0,NaN


In [367]:
t10.head()

,id,Cliente_ID,Producto_ID,d2,d3
cliprod,,,,,
4549769 32940,2,4549769,32940,2.0,NaN
4414012 35305,7,4414012,35305,NaN,NaN
397854 1240,8,397854,1240,4.0,NaN
4387996 2233,11,4387996,2233,2.0,2.0
4446449 1240,13,4446449,1240,4.0,4.0


In [368]:
s6 = s6.reset_index()
s6.head()

,cliprod,Demanda_uni_equil,d2,d3
0,1000001 1309,2,NaN,2.0
1,1000001 30572,4,NaN,NaN
2,1000001 43285,8,NaN,NaN
3,1000004 1238,1,NaN,1.0
4,1000004 1250,0,2.0,NaN


In [369]:
s6['Cliente_ID'], s6['Producto_ID'] = zip(*s6['cliprod'].apply(lambda x: x.split(' ', 1)))

In [370]:
s6.head()

,cliprod,Demanda_uni_equil,d2,d3,Cliente_ID,Producto_ID
0,1000001 1309,2,NaN,2.0,1000001,1309
1,1000001 30572,4,NaN,NaN,1000001,30572
2,1000001 43285,8,NaN,NaN,1000001,43285
3,1000004 1238,1,NaN,1.0,1000004,1238
4,1000004 1250,0,2.0,NaN,1000004,1250


In [371]:
s6 = s6.drop(['cliprod'], axis=1)
s6.head()

,Demanda_uni_equil,d2,d3,Cliente_ID,Producto_ID
0,2,NaN,2.0,1000001,1309
1,4,NaN,NaN,1000001,30572
2,8,NaN,NaN,1000001,43285
3,1,NaN,1.0,1000004,1238
4,0,2.0,NaN,1000004,1250


In [372]:
s7 = s7.reset_index()
s7['Cliente_ID'], s7['Producto_ID'] = zip(*s7['cliprod'].apply(lambda x: x.split(' ', 1)))
s7 = s7.drop(['cliprod'], axis=1)

s8 = s8.reset_index()
s8['Cliente_ID'], s8['Producto_ID'] = zip(*s8['cliprod'].apply(lambda x: x.split(' ', 1)))
s8 = s8.drop(['cliprod'], axis=1)

s9 = s9.reset_index()
s9['Cliente_ID'], s9['Producto_ID'] = zip(*s9['cliprod'].apply(lambda x: x.split(' ', 1)))
s9 = s9.drop(['cliprod'], axis=1)


t10 = t10.reset_index()
t10['Cliente_ID'], t10['Producto_ID'] = zip(*t10['cliprod'].apply(lambda x: x.split(' ', 1)))
t10 = t10.drop(['cliprod'], axis=1)

t11 = t11.reset_index()
t11['Cliente_ID'], t11['Producto_ID'] = zip(*t11['cliprod'].apply(lambda x: x.split(' ', 1)))
t11 = t11.drop(['cliprod'], axis=1)

In [373]:
t10.head()

,id,Cliente_ID,Producto_ID,d2,d3
0,2,4549769,32940,2.0,NaN
1,7,4414012,35305,NaN,NaN
2,8,397854,1240,4.0,NaN
3,11,4387996,2233,2.0,2.0
4,13,4446449,1240,4.0,4.0


In [374]:
train_joined = pd.concat([s6, s7, s8, s9], ignore_index = True)
test_joined = pd.concat([t10, t11], ignore_index = True)

In [375]:
train_joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41295709 entries, 0 to 41295708
Data columns (total 5 columns):
Demanda_uni_equil    int64
d2                   float64
d3                   float64
Cliente_ID           object
Producto_ID          object
dtypes: float64(2), int64(1), object(2)
memory usage: 1.8+ GB


In [376]:
train_joined.head()

,Demanda_uni_equil,d2,d3,Cliente_ID,Producto_ID
0,2,NaN,2.0,1000001,1309
1,4,NaN,NaN,1000001,30572
2,8,NaN,NaN,1000001,43285
3,1,NaN,1.0,1000004,1238
4,0,2.0,NaN,1000004,1250


In [377]:
train_joined[['Cliente_ID', 'Producto_ID']]=train_joined[['Cliente_ID', 'Producto_ID']].astype('int32')
test_joined[['Cliente_ID', 'Producto_ID']]=test_joined[['Cliente_ID', 'Producto_ID']].astype('int32')

In [378]:
train_joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41295709 entries, 0 to 41295708
Data columns (total 5 columns):
Demanda_uni_equil    int64
d2                   float64
d3                   float64
Cliente_ID           int32
Producto_ID          int32
dtypes: float64(2), int32(2), int64(1)
memory usage: 1.5 GB


In [379]:
test_joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6999251 entries, 0 to 6999250
Data columns (total 5 columns):
id             int64
Cliente_ID     int32
Producto_ID    int32
d2             float64
d3             float64
dtypes: float64(2), int32(2), int64(1)
memory usage: 267.0 MB


In [380]:
train_joined.fillna(value=0, inplace=True)
test_joined.fillna(value=0, inplace=True)

In [387]:
train_joined.to_csv('../input/train_d2d3.csv')
test_joined.to_csv('../input/test_d2d3.csv')

In [381]:
ids = test_joined['id']
test_joined = test_joined.drop(['id'],axis = 1)

y = train_joined['Demanda_uni_equil']
X = train_joined[test_joined.columns.values]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1729)

print ('Division_Set_Shapes:', X.shape, y.shape)
print ('Validation_Set_Shapes:', X_train.shape, X_test.shape)

('Division_Set_Shapes:', (41295709, 4), (41295709,))
('Validation_Set_Shapes:', (33036567, 4), (8259142, 4))


In [382]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33036567 entries, 32455689 to 38798989
Data columns (total 4 columns):
Cliente_ID     int32
Producto_ID    int32
d2             float64
d3             float64
dtypes: float64(2), int32(2)
memory usage: 1008.2 MB


In [383]:
params = {}
params['objective'] = "reg:linear"
params['eta'] = 0.025
params['max_depth'] = 5
params['subsample'] = 0.8
params['colsample_bytree'] = 0.6
params['silent'] = True

In [384]:
test_preds = np.zeros(test_joined.shape[0])
xg_train = xgb.DMatrix(X_train, label=y_train)
xg_test = xgb.DMatrix(X_test)

watchlist = [(xg_train, 'train_joined')]
num_rounds = 100

xgclassifier = xgb.train(params, xg_train, num_rounds, watchlist, feval = evalerror,
                         early_stopping_rounds= 20, verbose_eval = 10)
preds = xgclassifier.predict(xg_test, ntree_limit=xgclassifier.best_iteration)

print ('RMSLE Score:', rmsle(y_test, preds))

Will train until train_joined error hasn't decreased in 20 rounds.
[0]	train_joined-error:1.343162
[10]	train_joined-error:0.884865
[20]	train_joined-error:0.718050
[30]	train_joined-error:0.661067
[40]	train_joined-error:0.640778
[50]	train_joined-error:0.640144
[60]	train_joined-error:0.644474
Stopping. Best iteration:
[48]	train_joined-error:0.639296



('RMSLE Score:', 0.63985505117688801)


In [385]:
fxg_test = xgb.DMatrix(test_joined)
fold_preds = np.around(xgclassifier.predict(fxg_test, ntree_limit=xgclassifier.best_iteration), decimals = 1)
test_preds += fold_preds

In [ ]:
submission = pd.DataFrame({'id':ids, 'Demanda_uni_equil': test_preds})

In [388]:
submission.Demanda_uni_equil = submission.Demanda_uni_equil.round()

In [389]:
submission[["id","Demanda_uni_equil"]].to_csv('../submissions/' +
                                              datetime.now().strftime('%Y-%m-%d-%H-%M-%S') +'.csv', index=False)

print ('done')

done
